In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os
import logging
from parsel import Selector
from pprint import *
import time
from peewee import *
from playhouse.sqlite_ext import SqliteExtDatabase
import datetime
import re
import copy
import pandas as pd
from IPython.display import display, HTML, Image
import psycopg2
import json
from w3lib.url import url_query_parameter
import w3lib
"""xpath start here"""
xpath_pagination = """//div[@class="search-result"]/ul[@class="pagination"]"""
xpath_more_information_hover = """//ul[contains(@class, "result")]/li"""
xpath_next_page = """//div[@class="search-result"]/ul[@class="pagination"]/li[contains(@class, "volgende")][not(contains(@class, "not"))]"""
xpath_next_page_url = """//div[@class="search-result"]/ul[@class="pagination"]/li[contains(@class, "volgende")][not(contains(@class, "not"))]/a/@href"""
xpath_search_string_insert = """//*[(@id = "search-company-input")]"""
xpath_search_button = """/html/body/div[2]/main/div/div/div/div[2]/button"""
xpath_eight_digit_kvk = """//*[contains(concat( " ", @class, " " ), concat( " ", "meta", " " ))]//p[(((count(preceding-sibling::*) + 1) = 1) and parent::*)]/text()"""
xpath_twenty_digit_kvk = """//*[contains(concat( " ", @class, " " ), concat( " ", "result", " " ))]/@href"""
xpath_vesting_nr = """//*[contains(concat( " ", @class, " " ), concat( " ", "meta", " " ))]//p[(((count(preceding-sibling::*) + 1) = 2) and parent::*)]/text()"""
xpath_status_of_company = """//li/a/p[@class="status"]/text()"""
xpath_headquarters = """//*[contains(concat( " ", @class, " " ), concat( " ", "type", " " ))]/text()"""
xpath_main_company_name = """//h3/text()"""
xpath_bestaande_handelsnamen = """//div[@class="more-info"]/h4[contains(text(), "Bestaande")]/following-sibling::p/text()"""
xpath_statutaire_naam = """//div[@class="more-info"]/h4[contains(text(), "Statutaire")]/following-sibling::p/text()"""
xpath_second_page_link = """//*[contains(concat( " ", @class, " " ), concat( " ", "result", " " ))]/@href"""
"""xpath stop  here"""

'xpath stop  here'

In [2]:
global db
db = SqliteDatabase("kvk_temp.db")
db.connect()
class BaseModel(Model):
    class Meta:
        global db
        database = db

class kvkItemModel(BaseModel):
    search_string = CharField()
    eight_digit_kvk = CharField(null=True, unique=True)
    twenty_digit_kvk = CharField(null=True)
    vesting_nr = CharField(null=True)
    status_of_company = CharField(null=True)
    headquarters = BooleanField(null=True)
    website = CharField(null=True)
    main_company_name = CharField(null=True)
    bestaande_handelsnamen = CharField(null=True)
    statutaire_naam = CharField(null=True)
    second_page_company_name = CharField(null=True)
    street = CharField(null=True)
    housenumber = CharField(null=True)
    housenumber_extension = CharField(null=True)
    postcode = CharField(null=True)
    city = CharField(null=True)
    extra_infomation = CharField(null=True)
    second_page_link = CharField(null=True)
try:
    db.drop_tables([kvkItemModel])
except:
    pass
db.create_tables([kvkItemModel])
print("=" * 30)
print("Sqllite Intialization Succeed !")
print("=" * 30)
db.close()

Sqllite Intialization Succeed !


In [3]:
 def start_web_driver():
        # local browser        
        web_driver = webdriver.Chrome()
        web_driver.get("about:blank")
        return web_driver

#         # remote browser
#         web_driver = None
#         try:
#             web_driver = webdriver.Remote(command_executor="http://172.104.57.173:3000/wd/hub",
#                                              desired_capabilities=DesiredCapabilities.CHROME)
#             web_driver.get("about:blank")
#             return web_driver
#         except:
#             logging.error("Remote Chrome Webdriver failed")

#         raise Exception("There is no webdriver available for me")

driver = start_web_driver()

In [4]:
def read_post_codes_file():
    df = pd.read_csv("input.csv")
    return df["post_code"]

post_codes_list = read_post_codes_file()

In [5]:
def open_home_page():
    driver.get(url="about:blank")
    driver.get(url="https://www.kvk.nl/orderstraat/bedrijf-kiezen/?orig=#!shop?&q=")
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, xpath_search_string_insert))
        )
        print("=" * 30)
        print("Home page opened in remote browser !")
        print("=" * 30)
    except Exception as e:
        logging.error(e)
        print("Your ip may got banned, Check your internet connection please.")
        quit()
        
open_home_page()

Home page opened in remote browser !


In [6]:
input_post_code = post_codes_list[1]
total_page_of_postcode = 0

In [7]:
def insert_search_string_click_search():
    driver.find_element_by_xpath(xpath_search_string_insert).clear()
    driver.find_element_by_xpath(xpath_search_string_insert).send_keys(input_post_code)
    driver.find_element_by_xpath(xpath_search_button).click()
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, xpath_more_information_hover))
        )
        print("="*30)
        print("search string `%s` inserted !" % str(input_post_code))
        print("+"*30)
    except Exception as e:
        logging.error("Cannot insert search string to input field.")
        logging.error(e)

insert_search_string_click_search()

search string `1000AB` inserted !
++++++++++++++++++++++++++++++


In [8]:
def extract_data_from_page(total_page_of_postcode):
    total_page_of_postcode += 1
    companies = Selector(text=driver.page_source).xpath(xpath_more_information_hover).extract()
    print("=" * 30)
    print("collecting the %s page of postcode: %s" % (str(total_page_of_postcode), str(input_post_code)))
    print("found `%s` companies on this page" % len(companies))
    duplicate_count = 0
    valid_count = 0
    
    for company in companies:
        company_obj = Selector(text=company)
        item = kvkItemModel()
        item.search_string = copy.deepcopy(input_post_code)
        item.eight_digit_kvk = str(company_obj.xpath(xpath_eight_digit_kvk).extract_first()).strip().split()[1]
        item.twenty_digit_kvk = \
            str(company_obj.xpath(xpath_twenty_digit_kvk).extract_first()).strip().split('kvknummer=')[-1].split('&')[0]
        if len(item.twenty_digit_kvk) <= 11:
            item.twenty_digit_kvk = str(item.twenty_digit_kvk) + "0000"
        item.vesting_nr = str(company_obj.xpath(xpath_vesting_nr).extract_first()).strip().split(" ")[-1]
        if str(item.vesting_nr).startswith("Ves") is True:
            item.vesting_nr = "None"
        item.status_of_company = str(company_obj.xpath(xpath_status_of_company).extract_first()).strip()
        if company_obj.xpath(xpath_headquarters).extract_first() is None:
            item.headquarters = False
        else:
            item.headquarters = True
        item.main_company_name = str(company_obj.xpath(xpath_main_company_name).extract_first())
        item.bestaande_handelsnamen = str(company_obj.xpath(xpath_bestaande_handelsnamen).extract_first())
        item.statutaire_naam = str(company_obj.xpath(xpath_statutaire_naam).extract_first())
        item.second_page_link = "https://www.kvk.nl/" + str(
            company_obj.xpath(xpath_second_page_link).extract_first())
        try:
            item.save()
            valid_count += 1
            print("Valid Count:%s " % str(valid_count))
        except Exception as e:
            # This means we have inserted a duplicate record.
            duplicate_count += 1
            print("Duplicate Count:%s " % str(duplicate_count))
    
    print("+" * 30)
    
extract_data_from_page(total_page_of_postcode)

collecting the 1 page of postcode: 1000AB
found `10` companies on this page
Valid Count:1 
Valid Count:2 
Valid Count:3 
Valid Count:4 
Valid Count:5 
Valid Count:6 
Valid Count:7 
Duplicate Count:1 
Valid Count:8 
Duplicate Count:2 
++++++++++++++++++++++++++++++


In [10]:

driver.execute_script("$('body').css({'overflow':'hidden'}); $(document).bind('scroll',function () { window.scrollTo(0,document.body.scrollHeight); });")


In [11]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

In [9]:
def has_next_page():
    html = driver.page_source
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    driver.execute_async_script("$('body').css({'overflow':'hidden'});$(document).bind('scroll',function(){window.scrollTo(0,document.body.scrollHeight);});")
    if Selector(text=html).xpath(xpath_next_page).extract_first() is None:
        return False
    else:
        return True

In [10]:
def goto_next_page():
    element_to_click = driver.find_element_by_xpath(xpath_next_page)
#     driver.execute_script("arguments[0].scrollIntoView();", element_to_click)
    try:     
        actions = ActionChains(driver)
        actions.move_to_element(element_to_click)
        actions.perform()
        
        driver.find_element_by_xpath(xpath_next_page).click()
    except:
        goto_next_page()

In [11]:
def start_scrapping_all_result_pages():
    while has_next_page() is True:
        goto_next_page()
        extract_data_from_page(total_page_of_postcode)
    
    print("="*30)
    print("all the first page of postcode: `%s` finished collecting" % input_post_code)
    print("+"*30)

In [12]:
start_scrapping_all_result_pages()

TimeoutException: Message: Timed out


In [ ]:
e = EC.element_to_be_clickable((By.XPATH, xpath_next_page))

In [ ]:
e = driver.find_element_by_xpath(xpath_next_page)

In [27]:
e.is_selected()

False

In [31]:
ActionChains(driver).(driver.find_element_by_xpath(xpath_next_page)).perform()

MoveTargetOutOfBoundsException: Message: (306.45833587646484, 2158.10009765625) is out of bounds of viewport width (1011) and height (1039)


In [33]:
element_to_click = driver.find_element_by_xpath(xpath_next_page)
actions = ActionChains(driver)
driver.execute_script("arguments[0].scrollIntoView();", element)
actions.move_to_element(element)
actions.click(element_to_click)
actions.perform()